#### Test lstm implemtenation

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [3]:
torch.manual_seed(1)

In [4]:
torch.zeros(2)


 0
 0
[torch.FloatTensor of size 2]

In [3]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]


In [6]:
test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]


In [8]:
word_to_ix = {} 
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [9]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [10]:
class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        self.linear = nn.Linear(vocab_size, num_labels)

        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec))

In [11]:
# make a vector based on the index of a word
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


In [12]:
# tensor that contains the labels
def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [13]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)


In [14]:
for param in model.parameters():
    print(param)

Parameter containing:

Columns 0 to 9 
-0.0325  0.1950  0.0864  0.1697 -0.1961 -0.1459 -0.0775  0.1957 -0.1386 -0.1035
 0.1483 -0.1061 -0.1854  0.0135  0.0669  0.1624 -0.0324 -0.0168  0.0230 -0.0272

Columns 10 to 19 
-0.1599 -0.0406 -0.1231 -0.0440 -0.0606  0.0666 -0.0405  0.1708  0.0152  0.1358
-0.1411  0.1722 -0.1184  0.1092  0.1180  0.0847  0.1837  0.1188 -0.0732 -0.1597

Columns 20 to 25 
-0.0317 -0.0732  0.0726  0.0096 -0.1159 -0.0222
 0.0754  0.0071  0.1476  0.1432  0.1548  0.1291
[torch.FloatTensor of size 2x26]

Parameter containing:
-0.1628
 0.1293
[torch.FloatTensor of size 2]



In [15]:
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

Variable containing:
-0.8630 -0.5480
[torch.FloatTensor of size 1x2]



In [16]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [17]:
# Run on test data before we train, just to see a before-and-after
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-1.2611 -0.3332
[torch.FloatTensor of size 1x2]

Variable containing:
-1.1140 -0.3978
[torch.FloatTensor of size 1x2]



In [18]:
# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
-0.1599
-0.1411
[torch.FloatTensor of size 2]



In [19]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [20]:
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Variable as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()


In [21]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-0.2209 -1.6186
[torch.FloatTensor of size 1x2]

Variable containing:
-3.0979 -0.0462
[torch.FloatTensor of size 1x2]



In [22]:
# Index corresponding to Spanish goes up, English goes down!
print(next(model.parameters())[:, word_to_ix["it"]])

Variable containing:
-0.5788
 0.7577
[torch.FloatTensor of size 2]



http://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [35]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = open('dataset/test_file.txt', 'rb').read().decode('utf-8', 'ignore').split()

In [36]:

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['Hormone-mediated', 'maternal'], 'effects'), (['maternal', 'effects'], 'generate'), (['effects', 'generate'], 'variation')]
[
 28102.4844
[torch.FloatTensor of size 1]
, 
 26781.5430
[torch.FloatTensor of size 1]
, 
 25711.3359
[torch.FloatTensor of size 1]
, 
 24997.2129
[torch.FloatTensor of size 1]
, 
 24476.0488
[torch.FloatTensor of size 1]
, 
 24062.0391
[torch.FloatTensor of size 1]
, 
 23706.1055
[torch.FloatTensor of size 1]
, 
 23387.9922
[torch.FloatTensor of size 1]
, 
 23099.1406
[torch.FloatTensor of size 1]
, 
 22832.7812
[torch.FloatTensor of size 1]
]


In [114]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = open('dataset/test_file.txt', 'rb').read().decode('utf-8', 'ignore').split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    # the actual cbow vector is created here
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])
print(len(data))
print(vocab_size)
print(EMBEDDING_DIM)

[(['Hormone-mediated', 'maternal', 'generate', 'variation'], 'effects'), (['maternal', 'effects', 'variation', 'in'], 'generate'), (['effects', 'generate', 'in', 'offspring'], 'variation'), (['generate', 'variation', 'offspring', 'phenotype.'], 'in'), (['variation', 'in', 'phenotype.', 'In'], 'offspring')]
4078
1122
6


In [133]:
# EMBEDDING_SIZE = 300
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim): #, embedding_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                      embedding_dim=embedding_dim)
        
        #self.hidden = nn.Linear(in_features=embedding_dim, out_features=vocab_size)
        self.linear = nn.Linear(in_features=embedding_dim,
                                 out_features=vocab_size)
        
        
    def forward(self, x):
        # embeds 4 context words into say, 10 dim,
        # then take their sum along the rows (dim=0) to get 1 by 10 vector
        embedding = self.embedding(x).sum(dim=0).view((1,-1))
        # hidden = self.hidden(embedding)
        out = self.linear(embedding)
        out = F.log_softmax(out)
        return out

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
LEARNING_RATE = 0.001

losses = []
loss_function = nn.NLLLoss()
model = CBOW(vocab_size, EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(),lr=LEARNING_RATE)


In [134]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


make_context_vector(data[0][0], word_to_ix)  # example of the context of a word

Variable containing:
   90
 1088
  593
  511
[torch.LongTensor of size 4]

In [ ]:
for epoch in range(20000):
    total_loss = torch.Tensor([0])
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

In [109]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [44]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

In [45]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

In [46]:
loss_function = nn.NLLLoss()

In [48]:
LEARNING_RATE = 0.1
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [49]:
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

Variable containing:
-1.0310 -1.0991 -1.1706
-1.0382 -1.1287 -1.1318
-1.0352 -1.0843 -1.1818
-0.9920 -1.1524 -1.1607
-0.9313 -1.2333 -1.1563
[torch.FloatTensor of size 5x3]



In [78]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
        # print the loss
        if epoch % 30 == 0:
            print("Epoch %02d, loss = %f" % (epoch, loss.data[0] / len(training_data)))


Epoch 00, loss = 0.000498
Epoch 00, loss = 0.000611
Epoch 30, loss = 0.000494
Epoch 30, loss = 0.000606
Epoch 60, loss = 0.000490
Epoch 60, loss = 0.000600
Epoch 90, loss = 0.000485
Epoch 90, loss = 0.000595
Epoch 120, loss = 0.000482
Epoch 120, loss = 0.000590
Epoch 150, loss = 0.000478
Epoch 150, loss = 0.000584
Epoch 180, loss = 0.000474
Epoch 180, loss = 0.000579
Epoch 210, loss = 0.000470
Epoch 210, loss = 0.000574
Epoch 240, loss = 0.000466
Epoch 240, loss = 0.000569
Epoch 270, loss = 0.000463
Epoch 270, loss = 0.000564


In [79]:
# See what the scores are after training
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
# The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#  for word i. The predicted tag is the maximum scoring tag.
# Here, we can see the predicted sequence below is 0 1 2 0 1
# since 0 is index of the maximum value of row 1,
# 1 is the index of maximum value of row 2, etc.
# Which is DET NOUN VERB DET NOUN, the correct sequence!
print(tag_scores)


Variable containing:
-0.0097 -5.2097 -5.4770
-8.8351 -0.0004 -8.4203
-7.2420 -8.0797 -0.0010
-0.0015 -7.2341 -7.1892
-7.9857 -0.0005 -8.6537
[torch.FloatTensor of size 5x3]



In [ ]:
chars = []
for char, 